# Create RDR Network from OpenStreetMap (OSM) Data
This notebook walks through a few steps:

1. Download OSM data for a specified region
2. Convert OSM data to GMNS format using osm2gmns
3. Transform GMNS datasets to use RDR default specifications

### Import Modules

In [19]:
import os
import pandas as pd
import osm2gmns as og
from pathlib import Path

### Set Configuration/Defaults

In [20]:
# File path for the raw OSM data
# This path should be an existing OSM file (with .osm file extension) if already downloaded
# If the file path does not already exist, the tool will attempt to download the file based on relation ID defined below
input_osm_data = 'C:\GitHub\RDR\Data\osm_example\map.osm'

# Relation ID for OSM area
# This value is only used if above file path does not yet exist; otherwise can be set to 0
relation_id = 182725

# Directory for output GMNS data to be stored
# Output files are named link.csv and node.csv
output_dir = 'C:\GitHub\RDR\Data\osm_example\generated_files'

# Default value dictionaries
# OSM sometimes provides lanes and speed limits but any blanks for these columns, plus all capacity values and RDR facility types,
#   must be estimated based on functional class/link type
default_lanes_dict = {'motorway': 3, 'trunk': 2, 'primary': 2, 'secondary': 2, 'tertiary': 2,
                      'residential': 1, 'service': 1, 'cycleway': 1, 'footway': 1, 'track': 1,
                      'unclassified': 1, 'connector': 2}
default_speed_dict = {'motorway': 55, 'trunk': 45, 'primary': 40, 'secondary': 35, 'tertiary': 30,
                      'residential': 25, 'service': 30, 'cycleway': 5, 'footway': 5, 'track': 30,
                      'unclassified': 30, 'connector': 55}
default_capacity_dict = {'motorway': 2000, 'trunk': 1800, 'primary': 1200, 'secondary': 1000, 'tertiary': 800,
                         'residential': 500, 'service': 500, 'cycleway': 800, 'footway': 800, 'track': 800,
                         'unclassified': 800, 'connector': 9999}
default_factype_dict = {'motorway': '1', 'trunk': '2', 'primary': '3', 'secondary': '4', 'tertiary': '5',
                        'residential': '6', 'service': '7', 'cycleway': '7', 'footway': '7', 'track': '7',
                        'unclassified': '7', 'connector': '901'}

### Download OSM Data (manually or through osm2gmns)
There are several options to download raw OSM data that are described here: https://osm2gmns.readthedocs.io/en/latest/quick-start.html

1. As a first option, if a simple rectangular extent for your network is adequate, you can go to:
    - Go to https://www.openstreetmap.org/, zoom to your area, and try to export. The extracted file should have a .osm file extension.
    - Note that the default option may not be possible as it is limited to 50,000 nodes. In this case, try the second option.

2. The second option is to use the Overpass API directly via the downloadOSMData function in osm2gmns.
    - However, you will still need to go to https://www.openstreetmap.org/ to identify the unique relation id.
    - If OSM data is being downloaded via osm2gmns, the relation ID is needed to define the extent of the OSM extract. For example, the relation ID for Lima, Ohio can be found here: https://www.openstreetmap.org/relation/182725.

3. Geofabrik is a good third option to try because you can easily get extracts of specific geographies.
    - Website: https://download.geofabrik.de/
    - However, Geofabrik may not be ideal if your area of interest is small or overlaps multiple states.

In [ ]:
# Download OSM data as needed by relation_id
if os.path.exists(input_osm_data):
    pass
else:
    path = Path(input_osm_data)
    path.parent.absolute().mkdir(parents=True, exist_ok=True)
    og.downloadOSMData(relation_id, input_osm_data)

### Process Data Using OSM2GMNS
The osm2gmns package provides several methods to help convert the OSM data into the General Modeling Network Specification used by the RDR Tool Suite. The user should feel free to adjust the list of 'link_types' specified. For example, the user could remove 'residential' to make the RDR network simpler. Note that this helper tool only creates a road network for RDR.

The user should also feel free to adjust 'start_node_id' to accommodate the number of centroids in their network.

In [ ]:
# Process file into GMNS format
net = og.getNetFromFile(input_osm_data, network_types = ('auto'),
                        link_types = ['motorway','trunk', 'primary', 'secondary', 'tertiary', 'residential', 'unclassified'],
                        offset = 'right',
                        default_lanes = default_lanes_dict,
                        default_speed = default_speed_dict,
                        default_capacity = default_capacity_dict,
                        start_node_id = 100,
                        start_link_id = 1)

In [ ]:
# Combine short links
# If this code block produces an error for the user's region, it can be commented out
og.combineShortLinks(net)


In [ ]:
# Consolidate complex intersections
og.consolidateComplexIntersections(net, auto_identify=True)

In [ ]:
# Output the OSM network in GMNS CSV format
# Creates a GMNS_link.csv and GMNS_node.csv from the raw OSM data
og.outputNetToCSV(net, output_folder = output_dir)
os.rename(os.path.join(output_dir, 'link.csv'), os.path.join(output_dir, 'GMNS_link.csv'))
os.rename(os.path.join(output_dir, 'node.csv'), os.path.join(output_dir, 'GMNS_node.csv'))

### Prepare GMNS Files For RDR
#### node.csv
The node CSV file created by osm2gmns includes all of the RDR required columns: 'node_id', 'x_coord', 'y_coord', and 'node_type'.
  * However 'node_type' is typically blank, so below code labels all nodes as 'not_centroid'. Centroid nodes need to be separately created and appended to the file.
  * All node IDs are unique, per the RDR requirements.

#### link.csv
The link CSV created by osm2gmns includes the RDR required columns: 'link_id, 'from_node_id', 'to_node_id, 'length', 'capacity', 'free_speed', 'lanes', and 'allowed_uses'.
  * However, it does not include the following fields: 'directed' (has 'dir_flag' instead), 'facility_type' (has 'link_type' instead), 'toll', or 'travel_time'.
  * The below code: (1) converts 'dir_flag' to 'directed', (2) replaces 'auto' with 'c' in 'allowed_uses' column, (3) converts 'length' from meters to miles, (4) converts 'free_speed' from kmph to mph, (5) converts 'capacity' from veh/hr to veh/day/lane using a peak-hour-to-day conversion of 10.
  * The below code also creates a 'facility_type' column from 'link_type' based on the default value dictionary above.
  * The below code sets all toll values to zero. The user can manually add tolls the the output CSV file as needed.
  * The below code calculates 'travel_time' in minutes from 'length' and 'free_speed'.
  * Other fields that may be required depending on the scenario ('toll_nocar' and 'travel_time_nocar') are not included and need to be added manually by the user if needed.
  * All link IDs are unique, and 'link_id', 'from_node_id', and 'to_node_id' have no missing values, per the RDR requirements.

In [21]:
# Create node file
df_node = pd.read_csv(os.path.join(output_dir, 'GMNS_node.csv'),
                      skip_blank_lines=True,
                      converters={'node_id': str, 'node_type': str})
df_node['node_type'] = 'not_centroid'
df_node.to_csv(os.path.join(output_dir, 'node.csv'), index=False)

In [22]:
# Create link file
df_link = pd.read_csv(os.path.join(output_dir, 'GMNS_link.csv'),
                      skip_blank_lines=True,
                      converters={'link_id': str, 'from_node_id': str, 'to_node_id': str, 'dir_flag': int,
                                  'length': float, 'link_type_name': str, 'capacity': float, 'free_speed': float,
                                  'lanes': int, 'allowed_uses': str})
df_link['directed'] = df_link['dir_flag']
if (df_link['directed'] != 1).any():
    print("Warning: Not all links are one-way, a requirement for an RDR network. Manually adjust the link CSV file as needed.")
df_link['allowed_uses'] = 'c'
df_link['length'] = df_link['length'] * 0.000621371
df_link['free_speed'] = df_link['free_speed'] * 0.621371
df_link['capacity'] = 10 * df_link['capacity'] / df_link['lanes']
df_link['facility_type'] = df_link['link_type_name'].map(default_factype_dict)
df_link['toll'] = 0
df_link['travel_time'] = 60 * df_link['length'] / df_link['free_speed']
df_link.to_csv(os.path.join(output_dir, 'link.csv'), index=False)